# Build classification model: Sweet Asian and Indian Foods


## Introduction to classification: Clean, prep, and visualize your data

For dis four lessons, you go learn one of di main tins wey classic machine learning dey focus on - *classification*. We go waka through how to use different classification algorithms with one dataset about all di sweet cuisines wey dey Asia and India. Hope say you don dey hungry!

<p >
   <img src="../../../../../../translated_images/pinch.1b035ec9ba7e0d408313b551b60c721c9c290b2dd2094115bc87e6ddacd114c9.pcm.png"
   width="600"/>
   <figcaption>Celebrate pan-Asian cuisines for dis lessons! Image by Jen Looper</figcaption>

Classification na one kind [supervised learning](https://wikipedia.org/wiki/Supervised_learning) wey get plenty similarity with regression techniques. For classification, you go train one model to predict which `category` one item belong to. If machine learning na all about predicting values or names to tins by using datasets, then classification dey usually fall into two groups: *binary classification* and *multiclass classification*.

Make you remember:

-   **Linear regression** help you predict di relationship between variables and make correct predictions on where new datapoint go fall for di line. For example, you fit predict numeric values like *how much pumpkin go cost for September vs. December*.

-   **Logistic regression** help you find "binary categories": for dis price point, *di pumpkin orange or e no orange*?

Classification dey use different algorithms to find other ways to determine di label or class of one data point. Make we use dis cuisine data to see whether, by looking di group of ingredients, we fit know di cuisine origin.

### [**Pre-lecture quiz**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/19/)

### **Introduction**

Classification na one of di main work wey machine learning researchers and data scientists dey do. From basic classification of binary value ("dis email na spam or e no be spam?"), to complex image classification and segmentation using computer vision, e dey always useful to fit sort data into classes and ask questions about am.

To talk di process in one more scientific way, your classification method go create one predictive model wey go help you map di relationship between input variables and output variables.

<p >
   <img src="../../../../../../translated_images/binary-multiclass.b56d0c86c81105a697dddd82242c1d11e4d78b7afefea07a44627a0f1111c1a9.pcm.png"
   width="600"/>
   <figcaption>Binary vs. multiclass problems wey classification algorithms dey handle. Infographic by Jen Looper</figcaption>

Before we start to clean our data, visualize am, and prepare am for our ML tasks, make we first learn small about di different ways wey machine learning fit take classify data.

From [statistics](https://wikipedia.org/wiki/Statistical_classification), classification using classic machine learning dey use features, like `smoker`, `weight`, and `age` to determine *di chance to get X disease*. As one supervised learning technique wey be like di regression exercises wey you don do before, your data go get labels and di ML algorithms go use di labels to classify and predict classes (or 'features') of one dataset and assign dem to one group or outcome.

✅ Take small time to imagine one dataset about cuisines. Wetin one multiclass model fit answer? Wetin one binary model fit answer? Wetin if you wan know whether one particular cuisine dey likely to use fenugreek? Wetin if you wan see if, given one bag of star anise, artichokes, cauliflower, and horseradish, you fit create one typical Indian dish?

### **Hello 'classifier'**

Di question wey we wan ask from dis cuisine dataset na actually one **multiclass question**, because we get plenty national cuisines to work with. If dem give us one batch of ingredients, which of di many classes di data go fit enter?

Tidymodels get different algorithms wey you fit use to classify data, depending on di kind problem wey you wan solve. For di next two lessons, you go learn about some of dis algorithms.

#### **Prerequisite**

For dis lesson, we go need di following packages to clean, prepare and visualize our data:

-   `tidyverse`: Di [tidyverse](https://www.tidyverse.org/) na [collection of R packages](https://www.tidyverse.org/packages) wey dey make data science faster, easier and more fun!

-   `tidymodels`: Di [tidymodels](https://www.tidymodels.org/) framework na [collection of packages](https://www.tidymodels.org/packages/) for modeling and machine learning.

-   `DataExplorer`: Di [DataExplorer package](https://cran.r-project.org/web/packages/DataExplorer/vignettes/dataexplorer-intro.html) dey simplify and automate EDA process and report generation.

-   `themis`: Di [themis package](https://themis.tidymodels.org/) dey provide Extra Recipes Steps to handle Unbalanced Data.

You fit install dem like dis:

`install.packages(c("tidyverse", "tidymodels", "DataExplorer", "here"))`

Or, di script below go check whether you get di packages wey you need to complete dis module and go install dem for you if dem no dey.


In [ ]:
suppressWarnings(if (!require("pacman"))install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, DataExplorer, themis, here)

We go later load dis beta packages and make dem dey available for our current R session. (Dis na just for example, `pacman::p_load()` don already do am for you)


## Exercise - clean and balance your data

Di first work wey you go do before you start dis project na to clean and **balance** your data so you go fit get beta result.

Make we meet di data!🕵️


In [ ]:
# Import data
df <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/4-Classification/data/cuisines.csv")

# View the first 5 rows
df %>% 
  slice_head(n = 5)


Interestin! From wetin e be like, di first column na one kain `id` column. Make we get small more information about di data.


In [ ]:
# Basic information about the data
df %>%
  introduce()

# Visualize basic information above
df %>% 
  plot_intro(ggtheme = theme_light())

From di output, we fit see say we get `2448` rows and `385` columns and `0` missing values. We also get 1 discrete column, *cuisine*.

## Exercise - learn about cuisines

Now di work don dey more interesting. Make we check how di data dey spread for each cuisine.


In [ ]:
# Count observations per cuisine
df %>% 
  count(cuisine) %>% 
  arrange(n)

# Plot the distribution
theme_set(theme_light())
df %>% 
  count(cuisine) %>% 
  ggplot(mapping = aes(x = n, y = reorder(cuisine, -n))) +
  geom_col(fill = "midnightblue", alpha = 0.7) +
  ylab("cuisine")

E get limited number of food style (cuisine), but di way data dey spread no balance. You fit fix am! Before you do dat, try check am small.

Next, make we put each food style (cuisine) for im own tibble and find out how much data dey available (rows, columns) for each cuisine.

> A [tibble](https://tibble.tidyverse.org/) na modern data frame.

<p >
   <img src="../../../../../../translated_images/dplyr_filter.b480b264b03439ff7051232a8de1df9a8fd4df723db316feb4f9f5e990db4318.pcm.jpg"
   width="600"/>
   <figcaption>Artwork by @allison_horst</figcaption>


In [ ]:
# Create individual tibble for the cuisines
thai_df <- df %>% 
  filter(cuisine == "thai")
japanese_df <- df %>% 
  filter(cuisine == "japanese")
chinese_df <- df %>% 
  filter(cuisine == "chinese")
indian_df <- df %>% 
  filter(cuisine == "indian")
korean_df <- df %>% 
  filter(cuisine == "korean")


# Find out how much data is available per cuisine
cat(" thai df:", dim(thai_df), "\n",
    "japanese df:", dim(japanese_df), "\n",
    "chinese_df:", dim(chinese_df), "\n",
    "indian_df:", dim(indian_df), "\n",
    "korean_df:", dim(korean_df))

## **Exercise - Find di Top Ingredients for Each Cuisine wit dplyr**

Now, you fit dig deeper inside di data and sabi wetin be di common ingredients for each cuisine. You go need clean out di kind data wey dey cause wahala between cuisines, so make we learn about dis problem.

Create one function `create_ingredient()` for R wey go return one ingredient dataframe. Dis function go first remove one column wey no dey useful and arrange di ingredients based on how many times dem show.

Di basic structure of function for R be like dis:

`myFunction <- function(arglist){`

**`...`**

**`return`**`(value)`

`}`

If you wan sabi more about R functions, check dis [tidy introduction](https://skirmer.github.io/presentations/functions_with_r.html#1).

Make we start! We go use [dplyr verbs](https://dplyr.tidyverse.org/) wey we don dey learn for di previous lessons. As small reminder:

-   `dplyr::select()`: e dey help you choose which **columns** you wan keep or remove.

-   `dplyr::pivot_longer()`: e dey help you "stretch" di data, increase di number of rows and reduce di number of columns.

-   `dplyr::group_by()` and `dplyr::summarise()`: e dey help you find summary statistics for different groups, and arrange dem for better table.

-   `dplyr::filter()`: e dey create one small part of di data wey only get rows wey match your condition.

-   `dplyr::mutate()`: e dey help you create or change columns.

Check dis [*art*-filled learnr tutorial](https://allisonhorst.shinyapps.io/dplyr-learnr/#section-welcome) by Allison Horst, wey dey introduce some useful data wrangling functions for dplyr *(wey be part of Tidyverse)*.


In [ ]:
# Creates a functions that returns the top ingredients by class

create_ingredient <- function(df){
  
  # Drop the id column which is the first colum
  ingredient_df = df %>% select(-1) %>% 
  # Transpose data to a long format
    pivot_longer(!cuisine, names_to = "ingredients", values_to = "count") %>% 
  # Find the top most ingredients for a particular cuisine
    group_by(ingredients) %>% 
    summarise(n_instances = sum(count)) %>% 
    filter(n_instances != 0) %>% 
  # Arrange by descending order
    arrange(desc(n_instances)) %>% 
    mutate(ingredients = factor(ingredients) %>% fct_inorder())
  
  
  return(ingredient_df)
} # End of function

Now we fit use di function take sabi di top ten most popular ingredient by cuisine. Make we try am wit `thai_df`


In [ ]:
# Call create_ingredient and display popular ingredients
thai_ingredient_df <- create_ingredient(df = thai_df)

thai_ingredient_df %>% 
  slice_head(n = 10)

For di previous section, we use `geom_col()`, make we see how you fit use `geom_bar` too, to create bar charts. Use `?geom_bar` for more reading.


In [ ]:
# Make a bar chart for popular thai cuisines
thai_ingredient_df %>% 
  slice_head(n = 10) %>% 
  ggplot(aes(x = n_instances, y = ingredients)) +
  geom_bar(stat = "identity", width = 0.5, fill = "steelblue") +
  xlab("") + ylab("")

Make we do di same for di Japanese data.


In [ ]:
# Get popular ingredients for Japanese cuisines and make bar chart
create_ingredient(df = japanese_df) %>% 
  slice_head(n = 10) %>%
  ggplot(aes(x = n_instances, y = ingredients)) +
  geom_bar(stat = "identity", width = 0.5, fill = "darkorange", alpha = 0.8) +
  xlab("") + ylab("")


Wetin you mean about Chinese food?


In [ ]:
# Get popular ingredients for Chinese cuisines and make bar chart
create_ingredient(df = chinese_df) %>% 
  slice_head(n = 10) %>%
  ggplot(aes(x = n_instances, y = ingredients)) +
  geom_bar(stat = "identity", width = 0.5, fill = "cyan4", alpha = 0.8) +
  xlab("") + ylab("")

Make we check Indian food dem 🌶️.


In [ ]:
# Get popular ingredients for Indian cuisines and make bar chart
create_ingredient(df = indian_df) %>% 
  slice_head(n = 10) %>%
  ggplot(aes(x = n_instances, y = ingredients)) +
  geom_bar(stat = "identity", width = 0.5, fill = "#041E42FF", alpha = 0.8) +
  xlab("") + ylab("")

Finally, plot di Korean ingredients.


In [ ]:
# Get popular ingredients for Korean cuisines and make bar chart
create_ingredient(df = korean_df) %>% 
  slice_head(n = 10) %>%
  ggplot(aes(x = n_instances, y = ingredients)) +
  geom_bar(stat = "identity", width = 0.5, fill = "#852419FF", alpha = 0.8) +
  xlab("") + ylab("")

From di data visualizations, we fit drop di most common ingredients wey dey cause confusion between different cuisines, using `dplyr::select()`.

Everybody like rice, garlic and ginger!


In [ ]:
# Drop id column, rice, garlic and ginger from our original data set
df_select <- df %>% 
  select(-c(1, rice, garlic, ginger))

# Display new data set
df_select %>% 
  slice_head(n = 5)

## How to take care of data wey no balance ⚖️ - Use recipes 👩‍🍳👨‍🍳

<p >
   <img src="../../../../../../translated_images/recipes.186acfa8ed2e8f0059ce17ef22c9452d7b25e7e1e4b044573bacec9a18e040d2.pcm.png"
   width="600"/>
   <figcaption>Artwork by @allison_horst</figcaption>

Since dis lesson na about food, we go use `recipes` for di matter.

Tidymodels get one beta package wey dem call `recipes` - e dey help to arrange data well well.


Make we check how our food dey spread again.


In [ ]:
# Distribution of cuisines
old_label_count <- df_select %>% 
  count(cuisine) %>% 
  arrange(desc(n))

old_label_count

As you fit see, di number of cuisines no dey balance well. Korean cuisines dey almost 3 times Thai cuisines. Wen data no balance well, e fit affect how di model go perform. Make we think about binary classification. If most of di data dey one class, ML model go dey predict dat class more often, just because e get more data for am. To balance di data, we go adjust any data wey no dey balance well so dat e go remove di imbalance. Many models dey perform better wen di number of observations dey equal, and dem dey struggle wen di data no balance.

We get two main ways to handle data wey no balance:

-   Add more observations to di minority class: `Over-sampling` e.g use SMOTE algorithm

-   Remove some observations from di majority class: `Under-sampling`

Make we now show how to handle data wey no balance using `recipe`. Recipe na like blueprint wey dey describe di steps wey we go apply to di data set so e go ready for data analysis.


In [ ]:
# Load themis package for dealing with imbalanced data
library(themis)

# Create a recipe for preprocessing data
cuisines_recipe <- recipe(cuisine ~ ., data = df_select) %>% 
  step_smote(cuisine)

cuisines_recipe

Make we break down di steps wey we dey use for preprocessing.

-   Di call wey we make to `recipe()` wit one formula dey tell di recipe di *roles* wey di variables get, using `df_select` data as di reference. For example, di `cuisine` column don get `outcome` role, while di rest columns don get `predictor` role.

-   [`step_smote(cuisine)`](https://themis.tidymodels.org/reference/step_smote.html) dey create one *specification* of one recipe step wey dey generate new examples of di minority class synthetically, using nearest neighbors of di cases.

Now, if we wan see di preprocessed data, we go need [**`prep()`**](https://recipes.tidymodels.org/reference/prep.html) and [**`bake()`**](https://recipes.tidymodels.org/reference/bake.html) for our recipe.

`prep()`: e dey estimate di parameters wey we need from one training set, wey we fit later use for other data sets.

`bake()`: e dey take one prepped recipe and apply di operations to any data set.


In [ ]:
# Prep and bake the recipe
preprocessed_df <- cuisines_recipe %>% 
  prep() %>% 
  bake(new_data = NULL) %>% 
  relocate(cuisine)

# Display data
preprocessed_df %>% 
  slice_head(n = 5)

# Quick summary stats
preprocessed_df %>% 
  introduce()

Make we check how di food dem take spread and compare am wit di imbalanced data.


In [ ]:
# Distribution of cuisines
new_label_count <- preprocessed_df %>% 
  count(cuisine) %>% 
  arrange(desc(n))

list(new_label_count = new_label_count,
     old_label_count = old_label_count)

Yum! Di data dey nice and clean, e balance well, and e sweet die 😋!

> Normally, recipe na wetin dem dey use as preprocessor for modelling, e dey show di steps wey dem go apply for data set so e go ready for modelling. For dat kain case, na `workflow()` dem dey usually use (as we don already see for our previous lessons) instead of to dey estimate recipe by hand.
>
> So, you no go really need to dey use **`prep()`** and **`bake()`** recipes when you dey use tidymodels, but dem be useful tools wey you fit keep for your toolkit to confirm say di recipes dey do wetin you expect, like for our own case.
>
> When you **`bake()`** one prepped recipe with **`new_data = NULL`**, you go get di data wey you provide when you dey define di recipe back, but e go don pass through di preprocessing steps.

Make we save one copy of dis data now so we fit use am for future lessons:


In [ ]:
# Save preprocessed data
write_csv(preprocessed_df, "../../../data/cleaned_cuisines_R.csv")

Dis fresh CSV dey now for di root data folder.

**🚀Challenge**

Dis curriculum get plenti interestin datasets. Check di `data` folders and see if any dataset dey wey fit work for binary or multi-class classification. Wetin be di kind questions wey you go ask from dis dataset?

## [**Post-lecture quiz**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/20/)

## **Review & Self Study**

-   Go look [package themis](https://github.com/tidymodels/themis). Wetin be di other techniques wey we fit use to handle imbalanced data?

-   Tidy models [reference website](https://www.tidymodels.org/start/).

-   H. Wickham and G. Grolemund, [*R for Data Science: Visualize, Model, Transform, Tidy, and Import Data*](https://r4ds.had.co.nz/).

#### THANK YOU TO:

[`Allison Horst`](https://twitter.com/allison_horst/) for di amazing illustrations wey dey make R more friendly and fun. You fit find more illustrations for her [gallery](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).

[Cassie Breviu](https://www.twitter.com/cassieview) and [Jen Looper](https://www.twitter.com/jenlooper) for di original Python version of dis module ♥️

<p >
   <img src="../../../../../../translated_images/r_learners_sm.cd14eb3581a9f28d32086cc042ee8c46f621a5b4e0d59c75f7c642d891327043.pcm.jpeg"
   width="600"/>
   <figcaption>Artwork by @allison_horst</figcaption>


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Disclaimer**:  
Dis dokyument don use AI translet service [Co-op Translator](https://github.com/Azure/co-op-translator) do di translet. Even as we dey try make am correct, abeg make you sabi say machine translet fit get mistake or no dey accurate well. Di original dokyument for im native language na di one wey you go take as di correct source. For important mata, e good make you use professional human translet. We no go fit take blame for any misunderstanding or wrong interpretation wey fit happen because you use dis translet.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
